In [1]:
import pandas as pd

In [45]:
df = pd.read_excel(
    "dataset.xlsx",
    sheet_name="Table 1_rice",
    header=[4, 5]
)



df.columns = [
    '_'.join([str(i).strip() for i in col if 'Unnamed' not in str(i)])
    for col in df.columns
]

df = df[df['Region / Province'].notna()]

# Optionally drop rows that contain unwanted text
df = df[~df['Region / Province'].astype(str).str.contains('phase', case=False)]

df = df.iloc[:-6]

df

,Region / Province,"RICE, SPECIAL_Second Phase May 2025","RICE, SPECIAL_Second Phase May 2025.1","RICE, SPECIAL_First Phase Jun 2025","RICE, SPECIAL_First Phase Jun 2025.1","RICE, SPECIAL_Second Phase Jun 2025","RICE, SPECIAL_Second Phase Jun 2025.1","RICE, SPECIAL_Second Phase Jun 2025.2","RICE, WELL MILLED_Second Phase May 2025","RICE, WELL MILLED_Second Phase May 2025.1",...,"RICE, REGULAR MILLED_Second Phase Jun 2025.33","RICE, REGULAR MILLED_Second Phase Jun 2025.34","RICE, REGULAR MILLED_Second Phase Jun 2025.35","RICE, REGULAR MILLED_Second Phase Jun 2025.36","RICE, REGULAR MILLED_Second Phase Jun 2025.37","RICE, REGULAR MILLED_Second Phase Jun 2025.38","RICE, REGULAR MILLED_Second Phase Jun 2025.39","RICE, REGULAR MILLED_Second Phase Jun 2025.40","RICE, REGULAR MILLED_Second Phase Jun 2025.41","RICE, REGULAR MILLED_Second Phase Jun 2025.42"
2,Philippines,59.44,-,58.91,r,58.53,-,NaN,50.09,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NCR- National Capital Region,57.18,-,57,NaN,57.24,-,NaN,48.94,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CAR - Cordillera Administrative Region,58.81,-,57.62,r,57.62,-,NaN,51.71,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abra,59,-,58.5,NaN,58.5,-,NaN,56,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Apayao,51.5,-,51.5,r,51.5,-,NaN,37.96,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Lanao Del Sur,61.6,-,61.6,NaN,61.6,-,NaN,54.96,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,Maguindanao,47,-,46.5,NaN,45.75,-,NaN,43.75,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,Sulu,56.34,-,56.34,NaN,55.67,-,NaN,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,Tawi-Tawi,71.67,-,65.7,NaN,65.7,-,NaN,60,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np

# MELT: id_vars = the constant column
melted_df = pd.melt(
    df,
    id_vars=['Region / Province'],
    var_name='Commodity_Phase',
    value_name='Price'
)

# Split 'Commodity_Phase' into separate columns
melted_df[['Commodity', 'Phase']] = melted_df['Commodity_Phase'].str.split('_', n=1, expand=True)

# Drop the old combined column
melted_df = melted_df.drop(columns=['Commodity_Phase'])

# Reorder for clarity
melted_df = melted_df[['Region / Province', 'Commodity', 'Phase', 'Price']]


melted_df = melted_df.rename(columns={"Region / Province": "Province"})

melted_df[melted_df["Province"] == "Quezon"]


# Remove extra spaces within the 'Phase' values
melted_df["Phase"] = melted_df["Phase"].str.replace(r"\s+", " ", regex=True).str.strip()
melted_df["Phase"] = melted_df["Phase"].str.replace(r"\.\d+$", "", regex=True)

# Add a 'Date' column based on the 'Phase' values
def phase_to_date(phase):
    if phase == 'Second Phase May 2025':
        return '2025-05-30'
    elif phase == 'First Phase Jun 2025':
        return '2025-06-15'
    elif phase == 'Second Phase Jun 2025':
        return '2025-05-30'
    else:
        return np.nan

melted_df['Date'] = melted_df['Phase'].apply(phase_to_date)

# Drop averaged rows of regions
regions_to_drop = [
    "NCR", "CAR", "I", "II", "III", "IV-A", "MIMAROPA", "V", "VI", "NIR",
    "VII", "VIII", "IX", "X", "XI", "XII", "XIII", "BARMM"
]
pattern = r"^(" + "|".join(regions_to_drop) + r")\b"
melted_df = melted_df[~melted_df["Province"].str.strip().str.match(pattern)]


# Keep only rows where 'Price' can be converted to a float
melted_df = melted_df[melted_df['Price'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]


,Province,Commodity,Phase,Price,Date
0,Philippines,"RICE, SPECIAL",Second Phase May 2025,59.44,2025-05-30
3,Abra,"RICE, SPECIAL",Second Phase May 2025,59,2025-05-30
4,Apayao,"RICE, SPECIAL",Second Phase May 2025,51.5,2025-05-30
5,Benguet,"RICE, SPECIAL",Second Phase May 2025,55,2025-05-30
7,Kalinga,"RICE, SPECIAL",Second Phase May 2025,58,2025-05-30
...,...,...,...,...,...
2217,Basilan,"RICE, REGULAR MILLED",Second Phase Jun 2025,45,2025-05-30
2218,Lanao Del Sur,"RICE, REGULAR MILLED",Second Phase Jun 2025,50.34,2025-05-30
2219,Maguindanao,"RICE, REGULAR MILLED",Second Phase Jun 2025,39.09,2025-05-30
2221,Tawi-Tawi,"RICE, REGULAR MILLED",Second Phase Jun 2025,39.84,2025-05-30


In [54]:
melted_df[melted_df["Province"] == "Quezon"]

,Province,Commodity,Phase,Price,Date
35,Quezon,"RICE, SPECIAL",Second Phase May 2025,53,2025-05-30
269,Quezon,"RICE, SPECIAL",First Phase Jun 2025,53,2025-06-15
503,Quezon,"RICE, SPECIAL",Second Phase Jun 2025,53,2025-05-30
854,Quezon,"RICE, WELL MILLED",Second Phase May 2025,47,2025-05-30
1088,Quezon,"RICE, WELL MILLED",First Phase Jun 2025,47,2025-06-15
1322,Quezon,"RICE, WELL MILLED",Second Phase Jun 2025,46.5,2025-05-30
1673,Quezon,"RICE, REGULAR MILLED",Second Phase May 2025,42.5,2025-05-30
1907,Quezon,"RICE, REGULAR MILLED",First Phase Jun 2025,43,2025-06-15
2141,Quezon,"RICE, REGULAR MILLED",Second Phase Jun 2025,41,2025-05-30


In [55]:
# Output to csv
melted_df.to_csv("retail_price_cleaned.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'retail_price_cleaned.csv'